<a href="https://colab.research.google.com/github/ThesanR/TorontoPermitClassification/blob/main/PermitClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#2015 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/fbceeefc-a24d-43d0-8009-a1ec872550d0/download/Cleared%20Permits%202015.zip
!unzip -q "Cleared Permits 2015.zip"
#2016 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/b6f1907e-5a45-43f7-8c3c-56e4edc4f42f/download/Cleared%20Permits%202016.zip
!unzip -q "Cleared Permits 2016.zip"
#2017-2023 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/23de66da-ed1a-434a-a0fa-d85ec55f8f20/download/Cleared%20Building%20Permits%20since%202017.csv
#Active Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/108c2bd1-6945-46f6-af92-02f5658ee7f7/resource/dfce3b7b-4f17-4a9d-9155-5e390a5ffa97/download/building-permits-active-permits.csv

Classification of Permit applications from the City of Toronto between the years 2000 and 2023.

In [94]:
import pandas as pd
import numpy as np

df = pd.read_csv('clearedpermits2016.csv')
df = df.drop(df.columns[[1, 5, 6, 7, 8, 9, 10, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29]], axis=1)
df = df.dropna()
df = df.drop(df[(df.EST_CONST_COST == "0") | (df.EST_CONST_COST == "DO NOT UPDATE OR DELETE THIS INFO FIELD") | (df.STATUS != "Closed")].index)
print(df.STATUS.value_counts())
df


<ipython-input-94-0bf66b0cd84c>:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('clearedpermits2016.csv')


Closed    13036
Name: STATUS, dtype: int64


,PERMIT_NUM,PERMIT_TYPE,STRUCTURE_TYPE,WORK,WARD_GRID,APPLICATION_DATE,ISSUED_DATE,STATUS,EST_CONST_COST
0,00 119701 CMB,Residential Building Permit,SFD - Detached,Addition to Existing Building,W0508,2000/01/10,2000/04/19,Closed,38000
2,00 123349 CMB,Residential Building Permit,SFD - Detached,Addition/Alteration to Existing Building,W1307,2000/01/25,2000/01/25,Closed,3000
3,00 123820 CMB,Non-Residential Building Permit,Office,Alteration to Existing Building,N1503,2000/01/26,2000/03/28,Closed,11540
4,00 124475 PLB,Plumbing(PS),SFD - Detached,Install/Alter Plumbing - only,W1307,2000/01/31,2000/01/31,Closed,1000
6,00 124671 PLB,Plumbing(PS),SFD - Detached,Install/Alter Plumbing - only,W1303,2000/01/31,2000/02/09,Closed,100
...,...,...,...,...,...,...,...,...,...
41034,99 114871 CMB,Residential Building Permit,SFD Garages,New Building,S2105,1999/12/07,1999/12/07,Closed,10000
41040,99 117660 PLB,Plumbing(PS),Mixed Use/Res w Non Res,Install/Alter Plumbing - only,W0201,1999/12/22,2000/01/10,Closed,1000
41041,99 117935 PLB,Plumbing(PS),SFD - Semi-Detached,Install/Alter Plumbing - only,W1708,1999/12/23,1999/12/23,Closed,1000
41044,99 201382 000,Residential Building Permit,SFD - Detached,Addition to Existing Building,E4307,1999/07/08,1999/08/09,Closed,250000
